In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/properati_final.csv',error_bad_lines=False)

<IPython.core.display.Javascript object>

In [2]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['place_name']
le_barrio.fit(barrios)
properati['place_name'] = le_barrio.transform(barrios)

le_divisa = preprocessing.LabelEncoder()
divisa=properati['currency']
le_divisa.fit(divisa)
properati['currency'] = le_divisa.transform(divisa)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

In [3]:
cant_buckets = 100

#buckets
cantidad,rango = np.histogram(properati['price_per_m2'], bins=cant_buckets)
#properati['categories_by_price']=pd.cut(properati['price_per_m2'],rango,labels=np.arange(cant_buckets),include_lowest=True)
properati['categories_by_price']=pd.qcut(x=properati['price_per_m2'],q=cant_buckets,labels=np.arange(cant_buckets))

#properati['price_range']=pd.cut(properati['price_per_m2'],rango)

#casteo porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64)

properati.loc[:,('created_on')] = pd.to_datetime(properati['created_on'])
properati['anio'] = [x.year for x in properati['created_on'] ]

properati.groupby('categories_by_price').count().sort_values('price')

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,fecha,superficie_descubierta,anio
categories_by_price,,,,,,,,,,,,,,,,,,,,,
78.0,519,519,519,519,519,371,371,519,519,519,...,519,76,311,136,519,519,519,519,397,519
24.0,718,718,718,718,718,480,480,718,718,718,...,718,16,307,49,718,718,718,718,517,718
47.0,824,824,824,824,824,604,604,824,824,824,...,824,123,404,172,824,824,824,824,521,824
37.0,1127,1127,1127,1127,1127,771,771,1127,1127,1127,...,1127,82,503,170,1127,1127,1127,1127,731,1127
73.0,1161,1161,1161,1161,1161,910,910,1161,1161,1161,...,1161,208,695,303,1161,1161,1161,1161,831,1161
49.0,1235,1235,1235,1235,1235,929,929,1235,1235,1235,...,1235,174,646,295,1235,1235,1235,1235,764,1235
32.0,1241,1241,1241,1241,1241,837,837,1241,1241,1241,...,1241,77,570,145,1241,1241,1241,1241,806,1241
51.0,1249,1249,1249,1249,1249,912,912,1249,1249,1249,...,1249,192,635,292,1249,1249,1249,1249,743,1249
16.0,1257,1257,1257,1257,1257,672,672,1257,1257,1257,...,1257,13,423,31,1257,1257,1257,1257,1101,1257


In [8]:
properati.loc[properati['categories_by_price']!= properati['categories_by_price']].sort_values('price') #veo si clasifico una a categoria Nan

,created_on,property_type,place_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,floor,rooms,expenses,properati_url,description,title,fecha,superficie_descubierta,categories_by_price,anio
46159,2016-10-17,1,111,Capital Federal,"('-34.596814823,-58.3838596175',)",-34.596815,-58.383860,6000.0,1,1.058670e+05,...,NaN,NaN,9500.0,http://www.properati.com.ar/12crr_venta_depart...,CODIGO: 16 ubicado en: Marcelo T. de Alvear 10...,PISO DE ESTILO FRANCES con 6 dormitorios y 6 b...,"(2016, 10)",19.0,0.0,2016
97627,2014-03-25,2,490,Bs.As. G.B.A. Zona Norte,"('-34.480381,-58.6015434',)",-34.480381,-58.601543,6400.0,1,1.129248e+05,...,NaN,NaN,1200.0,http://www.properati.com.ar/egxy_venta_casa_tigre,CODIGO: 422-008 ubicado en: Corredor Bancalari...,"Casa en Venta en Barrio Privado Santa Barbara,...","(2014, 3)",NaN,0.0,2014
111727,2017-07-04,3,429,Bs.As. G.B.A. Zona Oeste,"('-34.6410586,-58.5598225',)",-34.641059,-58.559822,7500.0,1,1.323338e+05,...,NaN,NaN,NaN,http://www.properati.com.ar/19qp2_venta_local_...,"LOCALSARGENTO CABRAL 8, RAMOS MEJÍA. LOCAL DE ...",LOCAL EN VENTA,"(2017, 7)",NaN,7.0,2017
24508,2016-05-04,3,451,Bs.As. G.B.A. Zona Norte,"('-34.4410661,-58.5564334',)",-34.441066,-58.556433,8000.0,1,1.411560e+05,...,NaN,NaN,NaN,http://www.properati.com.ar/y7gh_venta_local_s...,LOCALAlquiler de Local y venta de fondo de com...,LOCAL EN VENTA,"(2016, 5)",NaN,2.0,2016
96968,2014-03-25,1,441,Capital Federal,"('-34.5491427288,-58.4849882126',)",-34.549143,-58.484988,9100.0,1,1.605650e+05,...,NaN,NaN,500.0,http://www.properati.com.ar/e8vu_venta_departa...,CODIGO: 611-789 ubicado en: CORREA 3650 - Pub...,"Departamento en Venta en Saavedra, Capital Fed...","(2014, 3)",NaN,4.0,2014
99528,2015-03-30,2,411,Bs.As. G.B.A. Zona Norte,"(nan,)",NaN,NaN,10000.0,1,1.764450e+05,...,NaN,NaN,NaN,http://www.properati.com.ar/n3hs_venta_casa_ma...,CONSULTAR PRECIO Casa de 325mts CERCA DE CANCH...,"Casa 2500m² con Plantas, Bs.As. G.B.A. Zona No...","(2015, 3)",2175.0,0.0,2015
26937,2016-06-24,2,493,Bs.As. G.B.A. Zona Norte,"(nan,)",NaN,NaN,10000.0,1,1.764450e+05,...,NaN,NaN,NaN,http://www.properati.com.ar/zgwo_venta_casa_to...,Casa totalmente reciclada - Tambien para alqui...,"Casa 1200m² con Dependencias, Malvinas Argenti...","(2016, 6)",798.0,0.0,2016
44897,2016-10-21,1,163,Bs.As. G.B.A. Zona Norte,"('-34.3308115,-58.7682009',)",-34.330811,-58.768201,11000.0,1,1.940895e+05,...,1.0,NaN,NaN,http://www.properati.com.ar/12nk3_venta_depart...,Lindo departamento en muy buena ubicaciÃ³n cer...,Departamento 49m² con Cochera en Avenida Gener...,"(2016, 10)",NaN,6.0,2016
132995,2017-07-20,2,294,Bs.As. G.B.A. Zona Norte,"('-34.4976342,-58.6608572',)",-34.497634,-58.660857,11000.0,1,1.940895e+05,...,NaN,NaN,NaN,http://www.properati.com.ar/1c2vd_venta_casa_i...,CODIGO: 772-vera ubicado en: marquez 5400 - P...,"Excelente Ubicacion , propiedad con gran lote","(2017, 7)",335.0,0.0,2017
54666,2016-12-07,1,518,Capital Federal,"('-34.6027525,-58.4269219',)",-34.602753,-58.426922,11000.0,1,1.940895e+05,...,NaN,NaN,NaN,http://www.properati.com.ar/1423w_venta_depart...,* EXCELENTE Depto de 2 amb A Estrenar * en EDI...,"CORRIENTES 4400 DOS AMBIENTES ESTRENAR ,VENTA","(2016, 12)",3.0,7.0,2016


In [5]:
#preparo set de datos
X = zip(properati['surface_total_in_m2'],properati['place_name'],properati['property_type'],properati['anio'])
y = properati['categories_by_price']

### Random Search

In [6]:
%%notify

from sklearn.model_selection import train_test_split
rf = RandomForestClassifier(n_jobs=-1)

max_features =["auto","sqrt","log2"]
n_estimators = np.arange(10,100,10)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"max_features":max_features,"n_estimators":n_estimators,}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=10,n_iter=3) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 304.48 segundos para 3 candidatos a hyper-parametros.

Puesto: 1
Promedio training score: 0.595 (std: 0.000)
Promedio validation score: 0.145 (std: 0.004)
Promedio fit time: 5.083s
Hiper-parametros: {'n_estimators': 70, 'max_features': 'log2'}

Puesto: 2
Promedio training score: 0.595 (std: 0.000)
Promedio validation score: 0.145 (std: 0.003)
Promedio fit time: 6.486s
Hiper-parametros: {'n_estimators': 90, 'max_features': 'log2'}

Puesto: 3
Promedio training score: 0.582 (std: 0.001)
Promedio validation score: 0.144 (std: 0.003)
Promedio fit time: 0.975s
Hiper-parametros: {'n_estimators': 10, 'max_features': 'sqrt'}



<IPython.core.display.Javascript object>

In [7]:
search.score(X_test,y_test) # pruebo el estimador refiteado con el set de test

0.15011733646230566